# Пример использования библиотеки gensim для тематического моделирования

**Корректность проверена на Python 3.6:**
+ gensim 3.4.0

In [1]:
from gensim import corpora, models

##  Данные 

### Формат UCI Bag of Words 

Из коллекции текстов удаляются лишние символы и проводится лемматизация (приведение к нормальной форме). Далее создаются словари слов и их частот в документах.

#### Список слов

In [11]:
print(*open("data/vocab.xkcd.txt").readlines()[:4], sep="")

boy
sits
barrel
floating



#### Частоты 

In [10]:
print(*open("data/docword.xkcd.txt").readlines()[:4], sep="")

1265
14352
87409
1 1 2



<pre>
Количество документов
Количество уникальных слов
Общее количество слов
Номер документа - Номер слова - Частота
...
</pre>

### Импортируем данные

In [2]:
data = corpora.UciCorpus("data/docword.xkcd.txt", "data/vocab.xkcd.txt")
dictionary = data.create_dictionary()

## Модель 

In [20]:
# Обучение модели
ldamodel = models.ldamodel.LdaModel(
                      corpus=data, 
                      id2word=dictionary, 
                      num_topics=5, 
                      passes=20, #количество проходов
                      # параметры распр. Дирихле
                      alpha=1.25, 
                      eta=1.25
                    )

In [20]:
# Сохранение модели
ldamodel.save("models/ldamodel_xkcd")

In [21]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("models/ldamodel_xkcd")

## Метрики качества 

In [29]:
# выводим топы слов
for topic, top_words in ldamodel.print_topics(num_topics=10, 
                                              num_words=3):
    print("Topic", topic, ":\n", top_words)

Topic 0 :
 0.002*"b'paul'" + 0.001*"b'ron'" + 0.001*"b'easy'"
Topic 1 :
 0.025*"b'man'" + 0.012*"b'text'" + 0.010*"b'title'"
Topic 2 :
 0.002*"b'wait'" + 0.002*"b'peter'" + 0.002*"b'sagal'"
Topic 3 :
 0.002*"b'text'" + 0.002*"b'map'" + 0.002*"b'title'"
Topic 4 :
 0.030*"b'person'" + 0.003*"b'text'" + 0.002*"b'girl'"


In [27]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print(2**(-perplexity))

368.5484096185736


In [31]:
perp = ldamodel.bound(data)
2**(-perp/float(87409))

368.548408452316

Перплексия должна измерятся на одном и том же объеме данных и числа тем. Её можно использовать только для сравнения моделей.

In [29]:
# Добавление в модель новых документов (дообучение), содержащихся в новом корупсе data2
ldamodel.update(data2, passes=10)

NameError: name 'data2' is not defined

In [30]:
# Получение распределения тем для конкретного документа
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.6622993),
 (1, 0.13572086),
 (2, 0.08688501),
 (3, 0.056953747),
 (4, 0.058141112)]